In [ ]:
import pandas as pd
import numpy as np

In [ ]:
news_df = pd.read_csv('сбербанк_2day(s)_news.csv')
news_df['date'] = pd.to_datetime(news_df['date'])

## Форматирование scraped даты

In [ ]:
months = {
    'январь': '01',
    'февраль': '02'
}

def process_date(date):
    possible_month = date.split()[0]
    for month in months.keys():
        if month in date:
            numerical_month = months.get(month)
            formatted_date = '-'.join(date.split()[0], numerical_month, date.split()[-1])
            return formatted_date
    

## Модель Microsoft для классификации

In [ ]:
import torch
from transformers import pipeline

device = 0 if torch.cuda.is_available() else -1
print('Видюха поддерживается!') if device == 0 else print('Видюха НЕ поддерживается :(')
classifier = pipeline(
    "zero-shot-classification", 
    model="facebook/bart-large-mnli", 
    device=device
)

In [ ]:
dir_out = 'sberbank_2d_microsoft_model_class.csv'

In [ ]:
candidate_labels = ["Financials & Dividends", "Strategy & Corporate Events", "Market Analysis & Expert Forecasts", "Macro & Regulation",  "Retail Products & Marketing", "Service & Tech Updates"]

def process_news_batched(df, batch_size=8):
    df = df.copy()
    texts = df['summary'].tolist()
    
    all_results = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        batch_results = classifier(
            batch_texts, 
            candidate_labels, 
            multi_label=False
        )
        
        if isinstance(batch_results, dict):
            batch_results = [batch_results]
            
        all_results.extend(batch_results)
        print(f"Обработано {min(i + batch_size, len(texts))}/{len(texts)}")

    for label in candidate_labels:
        df[label] = [
            res['scores'][res['labels'].index(label)] 
            for res in all_results
        ]
        
    return df

news_df = process_news_batched(news_df, batch_size=8) # Начните с 4 для 1060
news_df.to_csv(dir_out)

In [33]:
def extract_publisher(title):
    return title.split(' - ')[-1]

unique_pubs = news_df['title'].apply(extract_publisher)
print(f'уникальные издания: {np.unique(unique_pubs)}')

уникальные издания: ['110km.Ru' '1rnd.ru' '24 Канал' '29.ру' '360.ru' '365news.biz' '3DNews'
 '74.ру' '9111.ru' 'AKM.RU' 'Actualnews.org.' 'AdIndex.ru' 'Amic.ru'
 'AppleInsider.ru' 'Armenia News' 'BFM Кубань' 'BFM.ru' 'BK55' 'BUH.RU'
 'Bankinform.ru' 'Bankiros' 'CNews.ru' 'CRE.ru' 'CoinDesk' 'Coinspot.io'
 'Cryptopolitan' 'DKNews.kz' 'Daily Карелия' 'Deita.ru' 'Dela.ru'
 'Delo.ua' 'DigitalBusiness.kz' 'EAOMedia' 'Exclusive.kz' 'Finmarket.ru'
 'Finport.am' 'Finversia' 'Fonar.tv' 'Forbes.ru' 'Frank Media' 'Go31'
 'Gorodkirov.ru' 'Gus-info' 'INFOX.ru' 'InfoOrel.ru' 'InvestFuture'
 'Investing.com' 'Izhlife.ru' 'KGD.RU' 'KO44.ru' 'KONKURENT.RU' 'KU66'
 'KubanPress' 'Kursiv Media' 'MAAM.ru' 'Medvestnik' 'Metronews.ru'
 'Muksun.fm' 'MySlo' 'NEWS.ru' 'NewsTracker' 'Newsler.ru'
 'Novokuznetsk.su' 'PJSC Sberbank' 'PLUSworld' 'PNZ.RU' 'Pchela.News'
 'Peterburg2.ru' 'PopCornNews' 'Ppt.ru' 'PrimaMedia' 'Primpress.ru'
 'Privet-Rostov.ru' 'ProFinance' 'ProGorodNN' 'ProGorodNN.ru' 'RZN.info'
 'Radio1.

## Расчет Market Index

In [ ]:
''' словарь с весами изданий '''
source_weights = {
    # --- TIER 1: Максимальное влияние (Институционалы, Биржи, ГосСМИ) ---
    'Интерфакс': 2.0,
    'Интерфакс Россия': 1.8,
    'Московская Биржа': 2.0,
    'Ведомости': 1.8,
    'Forbes.ru': 1.7,
    'Сбербанк': 1.7,
    'PJSC Sberbank': 1.7,
    'ОАО «Сбер Банк': 1.5,
    'Альфа-Банк': 1.5,
    'БКС Экспресс': 1.5,
    'Финам.Ру': 1.5,
    'Коммерсантъ': 1.8, # (Если появится в списке)
    'ПРАВО.Ru': 1.4,
    'РАПСИ': 1.3,

    # --- TIER 2: Профильные финансы, Технологии и Рынки ---
    'Investing.com': 1.3,
    'Smart-Lab': 1.3,
    'ProFinance': 1.3,
    'Банки.ру': 1.2,
    'Эксперт': 1.2,
    'Frank Media': 1.2,
    'InvestFuture': 1.1,
    'Finmarket.ru': 1.1,
    'CNews.ru': 1.1,
    'Хабр': 1.1,
    'iXBT.com': 1.0,
    '3DNews': 1.0,
    'CoinDesk': 1.0,
    'Zakon.ru': 1.0,
    'BFM.ru': 1.1,
    'Клерк.ру': 1.0,

    # --- TIER 3: Крупные агрегаторы и Федеральные СМИ ---
    'ФОНТАНКА.ру': 1.0,
    'URA.RU': 1.0,
    'NEWS.ru': 0.9,
    'Lenta.ru': 0.9, # (Если появится)
    'Т—Ж': 0.9,
    't-j.ru': 0.9,
    'Лайфхакер': 0.8,
    'Аргументы и Факты': 0.8,
    'БИЗНЕС Online — Новости Казани': 0.9,
    'Независимая газета': 0.9,
    'ФедералПресс': 0.8,
    'SIA.RU': 0.8,
    'Сибирское информационное агентство': 0.8,

    # --- TIER 4: Заметные региональные и отраслевые СМИ ---
    'НГС.ру': 0.7,
    '74.ру': 0.7,
    '59.ру': 0.7,
    'NGS.42': 0.7,
    'Алтапресс — новости Барнаула и Алтайского края': 0.6,
    'PrimaMedia': 0.6,
    'SakhalinMedia': 0.6,
    'ЯСИА': 0.6,
    'Сибкрай.ru': 0.6,
    'Vremyan.ru': 0.5,
    'Время Н': 0.5,
    'Выберу.ру': 0.5,
    'ВсеЗаймыОнлайн': 0.4,
    'ADIndex.ru': 0.5,
    'Sostav.ru': 0.5,
    'AppleInsider.ru': 0.5,

    # --- TIER 5: Мелкие региональные порталы и шум ---
    # Для всех остальных устанавливаем базовый низкий вес
}

In [6]:
indicator_pipe = pipeline(task='text-classification', model='ProsusAI/finbert')

Loading weights: 100%|██████████| 201/201 [00:00<00:00, 1255.82it/s, Materializing param=classifier.weight]                                      
BertForSequenceClassification LOAD REPORT from: ProsusAI/finbert
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [53]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

def get_summary(text, max_sentences):
    if not text or len(text) < 100: return "Текст слишком короткий"
    
    # 1. ПРЕДВАРИТЕЛЬНАЯ ЧИСТКА
    # Заменяем переносы строк на пробелы, чтобы склеенные строки разошлись
    clean_text = text.replace("\n", " ").replace("\r", " ")
    
    # Убираем лишние пробелы (два и более подряд)
    clean_text = " ".join(clean_text.split())

    try:
        # 2. Sumy выбирает "лучший кусок" текста
        parser = PlaintextParser.from_string(clean_text, Tokenizer("russian"))
        summarizer = LexRankSummarizer()
        
        # Просим Sumy дать 1 предложение. 
        # Но если токенизатор ошибся, Sumy вернет огромный кусок.
        sumy_result = summarizer(parser.document, max_sentences)
        raw_summary = " ".join([str(s) for s in sumy_result])
        
        # 3. ФИНАЛЬНАЯ НАРЕЗКА (SAFETY NET)
        # Мы берем то, что вернула Sumy, и еще раз режем на предложения через NLTK
        real_sentences = sent_tokenize(raw_summary, language="russian")
        
        if real_sentences:
            # Возвращаем СТРОГО первое предложение
            return ' '.join(real_sentences[:max_sentences])
        else:
            return raw_summary
            
    except Exception as e:
        print(f"Ошибка в NLP: {e}")
        return "Ошибка обработки"

def calculate_sentiment_index(neutral, positive, negative):
    sentiment_index = (positive * 1.0) + (neutral * 0.5) + (negative * 0.0)
    return round(sentiment_index, 4)

def calculate_market_index(summary):
    for length in range(4, 1, -1):
        try:
            print(f'пробуем длину {length}')
            short_summary = get_summary(summary, max_sentences=length)
            result_all = indicator_pipe(short_summary, top_k=None)
            idx = calculate_sentiment_index(result_all[0]['score'], result_all[1]['score'], result_all[2]['score'])
            return idx
        except Exception as e:
            if hasattr(str(e), 'Token indices sequence length') or hasattr(str(e), 'The size of tensor'):
                print('ошибка длины, сжимаем...')
                continue

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\382he\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\382he\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [57]:
df_index = pd.read_csv(dir_out)
df_index['news_index'] = df_index[df_index["Retail Products & Marketing"] + df_index["Service & Tech Updates"] < 0.2]['summary'].apply(calculate_market_index)

пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 4
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 3
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 3
пробуем длину 4
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 3
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем длину 4
пробуем длину 4
пробуем длину 3
пробуем длину 2
пробуем 

#### взвешенный индекс

In [58]:
def weight_index(index, publisher):
    if index is not None:
        coeff = source_weights.get(publisher, 1)
        return index * coeff
    return None

for idx, row in df_index.iterrows():
    publisher = (row['title']).split(' - ')[-1]
    df_index.at[idx, 'news_index'] = weight_index(
        index=row['news_index'], 
        publisher=publisher
    )

In [59]:
df_index.set_index(df_index['date'], inplace=True)
df_index = df_index.drop(columns=['Unnamed: 0', 'date'])
df_index = df_index.sort_index()

In [60]:
df_index

,title,url,summary,Financials & Dividends,Strategy & Corporate Events,Market Analysis & Expert Forecasts,Macro & Regulation,Retail Products & Marketing,Service & Tech Updates,news_index
date,,,,,,,,,,
2026-01-01 08:00:00,Сбербанк ввел новую плату за обслуживание с 1 ...,https://finance.mail.ru/article/sberbank-vvel-...,Для большинства клиентов ежемесячная плата сос...,0.123660,0.155890,0.187873,0.205622,0.101408,0.225548,NaN
2026-01-01 08:00:00,Сбербанк ввёл новую комиссию с 1 января: кого ...,https://deita.ru/article/579663,С началом 2026 года у части клиентов Сбербанка...,0.130003,0.149978,0.200977,0.239756,0.093545,0.185741,NaN
2026-01-01 08:00:00,В Сбербанке прокомментировали запрет приема на...,https://pnz.ru/life/v-sberbanke-prokommentirov...,"Она сообщила, что привыкла жить «по старинке» ...",0.165612,0.162991,0.186449,0.193889,0.117371,0.173687,NaN
2026-01-01 08:00:00,Остался финальный дом: челябинские семьи выбир...,https://74.ru/text/longread/2026/01/01/76192683/,Экорайон «Вишневая горка» выбирают семьи с дет...,0.132006,0.167368,0.164031,0.285627,0.091916,0.159052,NaN
2026-01-01 08:00:00,С 1 января риск блокировки переводов увеличилс...,https://www.banki.ru/news/lenta/?id=11020500,C 1 января 2026 года Банк России обновил списо...,0.119717,0.184637,0.173481,0.254112,0.078264,0.189789,NaN
...,...,...,...,...,...,...,...,...,...,...
2026-02-20 17:57:05,Сбер и CHANGAN объединяют усилия: чем они пора...,https://110km.ru/art/changan-i-sberbank-podpis...,"20 февраля 2026, 20:54 CHANGAN и Сбербанк подп...",0.094162,0.187767,0.186657,0.230201,0.057411,0.243802,NaN
2026-02-20 18:18:49,Прогноз прибыли Сбера на 2026 год: рекордные п...,https://news.mondiara.com/categories/18/posts/...,Эксперты предсказали рост прибыли «Сбера» по и...,0.143432,0.147354,0.253696,0.205645,0.085381,0.164492,NaN
2026-02-20 19:08:18,"Обсуждение: ЦБ оштрафовал Сбербанк, Совкомбанк...",https://www.banki.ru/dialog/articles/43653/,Уже не первый раз Совкомбанк в период с 23:00 ...,0.132654,0.154584,0.171568,0.277559,0.092132,0.171502,NaN


## Подсчет Z-score индикатора

In [61]:
df_index.index = pd.to_datetime(df_index.index)
df_index['mean'] = df_index['news_index'].rolling(window='1D').mean()
df_index['std'] = df_index['news_index'].rolling(window='1D').std()
df_index['z'] = (df_index['news_index'] - df_index['mean']) / df_index['std']

## Визуализация - SBER IV, цена SBER vs INDEX (Z-score)

In [62]:
from moexalgo import Ticker, session
from datetime import timedelta
username = "382hejw@gmail.com"
password = "nTpM7b#N*wipF56"
session.authorize(username, password)

True

In [63]:
grp_index = df_index.groupby(pd.Grouper(freq='1D')).agg({'news_index': 'mean'})
sber = Ticker('SBER')

start_date = df_index.index.min().date()
end_date = df_index.index.max().date()

sber_candles = sber.candles(start=start_date, end=end_date, period='1d')
sber_candles['end'] = pd.to_datetime(sber_candles['end'])
options_df = pd.read_csv('sber_options_with_iv.csv')
options_df['date'] = pd.to_datetime(options_df['date'])

In [64]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots

# Строим график с двумя subplot'ами
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=('Курс акций Сбера', 'Новостной индикатор (Z-score)')
)

# Добавляем курс Сбера (без масштабирования, просто логарифм)
fig.add_trace(
    go.Scatter(
        x=sber_candles['end'], 
        y=sber_candles['close'], 
        mode='lines', 
        name='Цена акций Сбера',
        line=dict(color='blue', width=2),
        hovertemplate='Дата: %{x}<br>log(Цена): %{y:.3f}<extra>Цена акций</extra>'
    ),
    row=1, col=1
)

# Добавляем Z-score индикатор (без масштабирования, оригинальные значения)
fig.add_trace(
    go.Scatter(
        x=grp_index.index, 
        y=grp_index['news_index'].ffill(),  # используем оригинальный z-score
        mode='lines', 
        name='Новостной индикатор (Z-score)',
        line=dict(color='purple', width=2),
        hovertemplate='Дата: %{x}<br>Z-score: %{y:.3f}<extra></extra>'
    ),
    row=2, col=1
)

# Добавляем нулевую линию для Z-score (опционально)
fig.add_hline(
    y=0, 
    line_dash="dash", 
    line_color="gray", 
    opacity=0.5,
    row=2, col=1
)

# Улучшенный layout
fig.update_layout(
    title={
        'text': 'Сравнение курса Сбера и новостного индикатора',
        'x': 0.5,
        'xanchor': 'center',
        'font': dict(size=20)
    },
    hovermode='x unified',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor='rgba(255, 255, 255, 0.8)'
    ),
    template='plotly_white',
    width=1200,
    height=700  # немного увеличил высоту для двух графиков
)

# Настройка осей
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    ),
    row=2, col=1  # настройки для нижнего графика
)

# Настройка сетки для обоих графиков
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', row=1, col=1)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', row=2, col=1)
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', row=2, col=1)

# Добавляем подписи осей
fig.update_yaxes(title_text="Цена", row=1, col=1)
fig.update_yaxes(title_text="Z-score", row=2, col=1)
fig.update_xaxes(title_text="Дата", row=2, col=1)

fig.show()

In [65]:
print(df_index['z'].head(10))

date
2026-01-01 08:00:00   NaN
2026-01-01 08:00:00   NaN
2026-01-01 08:00:00   NaN
2026-01-01 08:00:00   NaN
2026-01-01 08:00:00   NaN
2026-01-01 08:00:00   NaN
2026-01-01 08:00:00   NaN
2026-01-01 08:00:00   NaN
2026-01-01 08:00:00   NaN
2026-01-01 08:00:00   NaN
Name: z, dtype: float64
